In [1]:
import pandas as pd

tr_sessions = pd.read_csv('../dataset/train_sessions_sorted.csv')
item_feat = pd.read_csv('../dataset/item_features.csv')
cand_items = pd.read_csv('../dataset/candidate_items.csv')
te_l = pd.read_csv('../dataset/test_leaderboard_sessions.csv')
te_f = pd.read_csv('../dataset/test_final_sessions.csv')
id2idx = pd.read_csv('./id2idx.csv')
tr_purchases = pd.read_csv('../dataset/train_purchases.csv')

tr_sessions = tr_sessions.merge(id2idx, how='inner', on='item_id')
tr_purchases = tr_purchases.merge(id2idx, how='inner', on='item_id')
te_l = te_l.merge(id2idx, how='inner', on='item_id')
te_f = te_f.merge(id2idx, how='inner', on='item_id')
cand_items = cand_items.merge(id2idx, how='inner', on='item_id')

In [5]:
print(item_feat['item_id'].nunique())
print(cand_items['item_id'].nunique())
print(tr_purchases.item_id.nunique())

x = set(tr_purchases.item_id.unique())
y = set(cand_items['item_id'].unique())
z = x.intersection(y)
print(len(z))
print(len(x-y))
print(len(y-x))


23691
4990
18907
4877
14030
113


In [10]:
print(tr_sessions.columns)
tr_sessions = tr_sessions[['session_id', 'item_id', 'date']]
cat = item_feat['feature_category_id'].unique()
val = item_feat['feature_value_id'].unique()
print(len(cat),  min(cat), max(cat))
print(len(val), min(val), max(val))
print(len(cat) * len(val))

Index(['session_id', 'Unnamed: 1', 'session_id.1', 'item_id', 'date'], dtype='object')
73 1 73
890 1 905
64970


### item id to list of feats to list of feat_id

In [56]:
[feat2ind[feat] for feat in item2feat[10]]

[0, 67, 47, 68, 69, 22, 70, 71, 55, 72, 4, 1, 73, 74, 75]

In [13]:
te_l = te_l.merge(id2idx, how='inner', on='item_id')
te_f = te_f.merge(id2idx, how='inner', on='item_id')
print(te_f.head(10))

   session_id  item_id                     date  item_idx_x  item_idx_y
0          61    27088  2021-06-01 08:12:39.664       22808       22808
1       64531    27088  2021-06-06 10:15:48.917       22808       22808
2      153529    27088   2021-06-03 09:19:30.91       22808       22808
3      153529    27088  2021-06-03 19:20:37.292       22808       22808
4      160053    27088  2021-06-25 09:30:36.342       22808       22808
5      199168    27088  2021-06-15 05:25:48.506       22808       22808
6      199168    27088  2021-06-15 05:26:18.928       22808       22808
7      217574    27088  2021-06-03 11:54:00.129       22808       22808
8      286801    27088  2021-06-19 21:18:44.012       22808       22808
9      327282    27088  2021-06-25 16:56:29.304       22808       22808


In [12]:
## Generate Train dataset txt file

sids = tr_purchases['session_id'].unique()

with open('./train_feat.txt', 'w') as f:
  for sid in sids:
    sess = tr_sessions.loc[tr_sessions['session_id'] == sid]['item_idx'].tolist()
    sess.append(tr_purchases.loc[tr_purchases['session_id'] == sid]['item_idx'].values[0])
    string = ",".join(map(str,sess)) + '\n'
    f.write(string)

KeyboardInterrupt: 

In [2]:
from sklearn.preprocessing import MultiLabelBinarizer

## check item id size
print(item_feat['item_id'].nunique())
# Create MultiLabelBinarizer object
mlb = MultiLabelBinarizer()
# One-hot encode data
##categorical
cat = item_feat.groupby("item_id")['feature_category_id'].apply(list)
print(cat.head())
print(mlb.fit_transform(cat))
multihot_cat = mlb.fit_transform(cat)
print(multihot_cat.shape)
print(list(mlb.classes_))

##value
val = item_feat.groupby("item_id")['feature_value_id'].apply(list)
print(val.head())
print(mlb.fit_transform(val))
multihot_val = mlb.fit_transform(val)
print(multihot_val.shape)
print(list(mlb.classes_))



23691
item_id
2    [56, 62, 68, 33, 72, 29, 16, 50, 61, 53, 7, 69...
3    [56, 69, 68, 17, 32, 11, 45, 7, 19, 46, 61, 73...
4    [55, 17, 5, 69, 18, 68, 50, 73, 65, 7, 59, 47,...
7    [56, 50, 63, 41, 26, 32, 61, 68, 69, 7, 15, 44...
8    [56, 55, 7, 69, 72, 3, 45, 18, 59, 73, 5, 61, ...
Name: feature_category_id, dtype: object
[[0 0 0 ... 0 1 0]
 [0 0 1 ... 0 1 1]
 [0 0 1 ... 0 1 1]
 ...
 [1 0 1 ... 0 1 1]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 1]]
(23691, 73)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]
item_id
2    [365, 801, 351, 802, 75, 123, 38, 76, 462, 6, ...
3    [365, 592, 14, 378, 902, 859, 559, 452, 254, 8...
4    [267, 378, 605, 538, 289, 373, 317, 544, 521, ...
7    [153, 708, 816, 759, 268, 902, 706, 739, 592, ...
8    [365, 267, 798, 592, 

In [24]:
import numpy as np
concat = np.hstack((multihot_cat, multihot_val))

print(concat.shape)
np.save("concat.npy", concat)

NameError: name 'multihot_cat' is not defined

In [3]:
import numpy as np
concat = np.load("../processed/concat.npy")
print(concat[10:,], concat.shape)


[[0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] (23691, 963)


In [14]:
arr = np.array([0]*23691)
for k,v in purid2idx.items():
  # print(k,v)
  arr[v] += 10
for k,v in cand_idx.items():
  arr[v] += 10
arr = arr[np.newaxis]
arr = arr.transpose()
print(arr)

concat = np.hstack((concat, arr))
print(concat.shape)

[[ 0]
 [10]
 [20]
 ...
 [10]
 [ 0]
 [10]]
(23691, 964)


In [6]:
num_items = concat.shape[0]
A = np.zeros((num_items, num_items))
print(A)
from tqdm import tqdm
with open('./train.txt', 'r') as f: 
  for line in tqdm(f.readlines()): # idx
    # print(line)
    items = list(map(int, line.split(",")))
    # print(len(items))
    for i in range(len(items)-1):
      # print(i)
      A[items[i]][items[i+1]] += 1
      A[items[i+1]][items[i]] += 1

from sklearn.preprocessing import normalize
normed_matrix = normalize(A, axis=1, norm='l1')
# [[ 0.          0.33333333  0.66666667]
#  [ 0.25        0.33333333  0.41666667]
#  [ 0.28571429  0.33333333  0.38095238]]

normed_matrix

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


100%|██████████| 1000000/1000000 [00:13<00:00, 74971.80it/s]


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03669725, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.04793609, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.03003003, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.09090909]])

In [9]:
np.save("norm_A", normed_matrix)
np.save("A", A)

In [10]:
print(normed_matrix[8122][8122])
print(A[8122][8122])

0.08035714285714286
18.0


# ItemID to idx

In [5]:
id2idx = {}
for i, item_id in enumerate(item_feat.item_id.unique()):
  id2idx[item_id] = i
print(len(id2idx.keys()))
id2idx_df = pd.DataFrame(list(id2idx.items()),columns = ['item_id','item_idx'])
id2idx_df.to_csv('./id2idx.csv', index=False)




23691


In [11]:
## purchased item id to idx
tr_purchases = pd.read_csv('../dataset/train_purchases.csv')
purid2idx = {}
for i, iid in enumerate(tr_purchases.item_id.unique()):
  purid2idx[iid] = id2idx[iid]
purid2idx_df = pd.DataFrame(list(purid2idx.items()),columns = ['item_id','pur_idx'])
purid2idx_df.to_csv('./purid2idx.csv', index=False)

In [21]:
pd.merge(left=id2idx_df, right=purid2idx, how='inner', on='item_id')

,item_id,item_idx,pur_idx
0,3,1,10900
1,4,2,13214
2,7,3,18530
3,8,4,9694
4,9,5,15648
...,...,...,...
18902,28138,23685,10832
18903,28139,23686,84
18904,28140,23687,6491
18905,28141,23688,6688


In [13]:
## candidate item id to idx
cand = pd.read_csv('../dataset/candidate_items.csv')

cand_idx = {}
for i, iid in enumerate(cand.item_id.unique()):
  cand_idx[iid] = id2idx[iid]
candid2idx_df = pd.DataFrame(list(cand_idx.items()),columns = ['item_id','idx'])
candid2idx_df.to_csv('./candid2idx_df.csv', index=False)

In [13]:
print(tr_purchases.item_id.nunique())

18907


# Item idx to purchase &candidate idx

In [8]:
## purchased item id to idx
tr_purchases = pd.read_csv('../dataset/train_purchases.csv')
## candidate item id to idx
cand = pd.read_csv('../dataset/candidate_items.csv')
candpurid2idx = {}
i=0
for i, iid in enumerate(tr_purchases.item_id.unique()):
  candpurid2idx[iid] = i
  i+=1
for i, iid in enumerate(cand.item_id.unique()):
  candpurid2idx[iid] = i
  i+=1
candpurid2idx_df = pd.DataFrame(list(candpurid2idx.items()),columns = ['item_id','pur_idx'])
candpurid2idx_df.to_csv('./purcand2idx_df.csv', index=False)

In [9]:
candpurid2idx_df.shape

(19020, 2)

## SID to bought item id

In [7]:
sid2iid = {}
tr_purchases = pd.read_csv('../dataset/train_purchases.csv')
print(tr_purchases.head())
for i, row in tr_purchases.iterrows():
  sid2iid[row['session_id']] = row['item_id']

   session_id  item_id                     date
0           3    15085  2020-12-18 21:26:47.986
1          13    18626  2020-03-13 19:36:15.507
2          18    24911  2020-08-26 19:20:32.049
3          19    12534   2020-11-02 17:16:45.92
4          24    13226  2020-02-26 18:27:44.114


## Convert current trpur_sessions['item_id'] -> multilabel features

In [6]:

trpur_sessions = pd.read_csv('../dataset/trpur_sessions.csv')
print(trpur_sessions.columns)
trpur_sessions['feat'] = trpur_sessions.apply(lambda row: concat[id2idx[row['item_id']]], axis=1)
trpur_sessions['pur'] = trpur_sessions.apply(lambda row: id2idx[sid2iid[row['session_id']]], axis=1)
print(trpur_sessions.head())


Index(['Unnamed: 0', 'session_id', 'item_id', 'date'], dtype='object')


KeyError: 9655

In [4]:
## Decompsition 9xx -> ??
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import FactorAnalysis

print(item_feat.item_id.nunique())

def dimensionality_reduction(feat, y, kind='lda'):
  if kind=='lda':
    model = LinearDiscriminantAnalysis()
  elif kind=='pca':
    model = PCA()
  elif kind=='kernelpca':
    model = KernelPCA()
  elif kind=='truncatedsvd':
    model = TruncatedSVD()
  elif kind=="FA":
    n_components=10
    model = FactorAnalysis(n_components=n_components, random_state=0)
  
    new_feat = model.fit_transform(feat, y)
    print("new feat shape", new_feat.shape)
    np.save("../processed/{}_{}_feat.npy".format(str(kind),n_components), new_feat)
  return new_feat



23691


In [7]:
# trpur_feat = trpur_sessions['feat'].to_list()

dimensionality_reduction(concat, None, 'FA')

new feat shape (23691, 10)


array([[ 1.01354743e+00, -2.36522428e-01, -1.78191219e-01, ...,
        -1.92526866e+00,  4.54186979e-02,  1.46163958e-01],
       [-9.04081155e-01, -2.62624712e-01,  1.90069086e-02, ...,
         3.56022303e-04, -1.32319546e+00,  3.95381329e-02],
       [-9.04081155e-01, -2.62624712e-01,  1.90069086e-02, ...,
         3.33666356e-04, -1.33722074e+00, -3.97352781e-01],
       ...,
       [-1.07802177e+00, -3.72028055e-01,  4.30769278e-02, ...,
         5.60497418e-04, -3.23578789e-01, -7.00453048e-01],
       [-9.04081155e-01, -2.62624712e-01,  1.90069086e-02, ...,
        -1.46177380e-03,  2.09284745e+00,  7.71320903e-01],
       [-9.04081155e-01, -2.62624712e-01,  1.90069086e-02, ...,
         2.25556626e-04, -3.45470278e-01,  2.23610558e+00]])

In [33]:
trpur_feat = trpur_sessions['feat'].to_list()
y = trpur_sessions['pur'].to_list()
lda_feat = dimensionality_reduction(trpur_feat, y, 'lda')

In [35]:
pca_feat = dimensionality_reduction(trpur_feat, y, 'pca')


new feat shape (5743820, 963)


In [36]:
kernelpca_feat = dimensionality_reduction(trpur_feat, y, 'kernelpca')
tsvd_feat = dimensionality_reduction(trpur_feat, y, 'truncatedsvd')

MemoryError: Unable to allocate 240. TiB for an array with shape (5743820, 5743820) and data type float64

## play around embedding

In [ ]:
import torch
import torch.nn as nn
weight = torch.FloatTensor(concat)
embedding = nn.Embedding.from_pretrained(weight, freeze=False, max_norm=1, norm_type=2.0)
input = torch.LongTensor([1])
print(embedding(input))
print(embedding(input).norm(dim=-1))
embedding.weight
embedding(input).mean().backward()

In [22]:
emb_bag = nn.EmbeddingBag.from_pretrained(weight, freeze=False, max_norm=1, norm_type=2.0, )
input = torch.LongTensor([1])
offset = torch.LongTensor([0])
print(emb_bag(input, offset))
print(emb_bag(input, offset).norm(dim=-1))
indice = torch.LongTensor([0])
print(emb_bag(indice, offset))
indices = torch.LongTensor([0,1])
print(emb_bag(indices, offset))
emb_bag.weight
emb_bag(input, offset).mean().backward()

tensor([[0.0000, 0.0000, 0.1443, 0.1443, 0.1443, 0.0000, 0.1443, 0.0000, 0.0000,
         0.0000, 0.1443, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1443, 0.1443,
         0.1443, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1443, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.1443, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1443,
         0.1443, 0.1443, 0.0000, 0.0000, 0.1443, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1443, 0.1443, 0.0000, 0.0000, 0.1443, 0.0000, 0.1443, 0.0000, 0.1443,
         0.0000, 0.1443, 0.0000, 0.0000, 0.1443, 0.1443, 0.0000, 0.0000, 0.1443,
         0.1443, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.1443, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [24]:
indices=torch.LongTensor([[1,0]])
print(emb_bag(indices))

tensor([[0.0000, 0.0000, 0.0722, 0.0722, 0.0722, 0.0000, 0.1722, 0.0000, 0.0000,
         0.0000, 0.0722, 0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.0722, 0.0722,
         0.0722, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0722, 0.0000,
         0.0000, 0.1000, 0.0000, 0.0000, 0.0722, 0.1000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0722,
         0.0722, 0.1722, 0.0000, 0.0000, 0.1722, 0.0000, 0.0000, 0.1000, 0.0000,
         0.0722, 0.1722, 0.0000, 0.0000, 0.0722, 0.0000, 0.1722, 0.1000, 0.0722,
         0.0000, 0.0722, 0.0000, 0.0000, 0.1722, 0.1722, 0.0000, 0.0000, 0.1722,
         0.0722, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0722, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1000, 0.0

In [1]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
FA_128_feat = t.FloatTensor(np.load('./FA_128_feat.npy'))
normed_matrix = t.FloatTensor(np.load('./norm_A.npy'))
in_feat = 128
out_feat = 19021
device = t.device('cuda')

/home/seongha/anaconda3/envs/serec/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Net(nn.Module):
    def __init__(self, in_feat, out_feat):
      super(Net, self).__init__()

      self.fc1 = nn.Linear(in_feat, out_feat)
      self.dropout1 = nn.Dropout2d(0.25)
      # self.fc2 = nn.Linear(523, 19021)

    def forward(self, x):
      x = self.fc1(x)
      x = t.relu(x)

      # Apply softmax to x
      output = F.log_softmax(x, dim=1)
      return output


my_nn = Net(in_feat, out_feat).to(device)
print(my_nn)

Net(
  (fc1): Linear(in_features=128, out_features=19021, bias=True)
  (dropout1): Dropout2d(p=0.25, inplace=False)
)


In [3]:
num_items = 23691
from tqdm import tqdm

class DataLoader:
  def __init__(self, fn, bs):
    self.fn = fn
    self.bs = bs
  def __iter__(self):
    with open(self.fn, 'r') as f: 
      for line in tqdm(f.readlines()): # idx
        # print(line)
        train = np.zeros((num_items, num_items))
        label = np.zeros(num_items)
        items = list(map(int, line.split(",")))
        # print(len(items))
        for i in range(len(items)-2):
          # print(i)
          train[items[i]][items[i+1]] = 1
          train[items[i+1]][items[i]] = 1
        label[items[-1]] = 1
        yield train, label

In [5]:
class BPRLoss(nn.Module):
    def __init__(self):
        super(BPRLoss, self).__init__()

    def forward(self, logit):
        """
        Args:
            logit (BxB): Variable that stores the logits for the items in the mini-batch
                         The first dimension corresponds to the batches, and the second
                         dimension corresponds to sampled number of items to evaluate
        """
        # differences between the item scores
        diff = logit.diag().view(-1, 1).expand_as(logit) - logit
        # final loss
        loss = -torch.mean(F.logsigmoid(diff))
        return loss


class BPR_max(nn.Module):
    def __init__(self):
        super(BPR_max, self).__init__()
    def forward(self, logit):
        logit_softmax = F.softmax(logit, dim=1)
        diff = logit.diag().view(-1, 1).expand_as(logit) - logit
        loss = -torch.log(torch.mean(logit_softmax * torch.sigmoid(diff)))
        return loss

loss_func = BPR_max()

optim = t.optim.Adam(my_nn.parameters(), lr = 0.01, weight_decay = 0)

In [4]:
loss_func = nn.CrossEntropyLoss()

In [6]:
trainloader = DataLoader('./train.txt', 1)
AX = t.matmul(normed_matrix, FA_128_feat)
for i, (sess, label) in tqdm(enumerate(trainloader)):
  # print(sess,label)
  sess = t.FloatTensor(sess)
  sess = t.matmul(sess, AX).to(device) ## 23691 * 128
  label = t.FloatTensor(label).to(device)
  # label = t.FloatTensor(label).to(device)
  
  logit = my_nn(sess)
  loss = loss_func(logit, label)
  print(loss)
  loss.backward()
  optim.step()
  del sess
  del label


0it [00:01, ?it/s]
  0%|          | 0/1000000 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 1.68 GiB (GPU 0; 9.78 GiB total capacity; 5.07 GiB already allocated; 425.56 MiB free; 5.08 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF